# Preprocessing and clustering functions

This tutorial explores preprocessing and clustering

## Scatter plots for embeddings

With scanpy, scatter plots for tSNE, UMAP and several other embeddings are readily available using the `sc.pl.tsne`, `sc.pl.umap` etc. functions. See [here](https://scanpy.readthedocs.io/en/stable/api/scanpy.plotting.html#embeddings) the list of options.

Those functions access the data stored in `adata.obsm`. For example `sc.pl.umap` uses the information stored in `adata.obsm['X_umap']`. For more flexibility, any key stored in `adata.obsm` can be used with the generic function `sc.pl.embedding`.

In [ ]:
import scanpy as sc
import pandas as pd
from matplotlib.pyplot import rc_context
import scipy
import anndata
import os
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
sc.set_figure_params(dpi=100, color_map = 'viridis_r')
sc.settings.verbosity = 3
sc.logging.print_header()

In [ ]:
# configuration for capsule
from pathlib import Path

sc._settings.ScanpyConfig.cachedir = Path("/data/cache/")
sc._settings.ScanpyConfig.datasetdir = Path("/data/")
Path("/results/figures").mkdir(parents=True, exist_ok=True)
sc._settings.ScanpyConfig.figdir = Path("/results/figures")
Path("/results/write").mkdir(parents=True, exist_ok=True)
sc._settings.ScanpyConfig.writedir = Path("/results/write/")

In [ ]:
results_dir = "/results"
results_file = (
    f"{results_dir}/write/INCF.h5ad"  # the file that will store the analysis results
)

### Load dataset

In [ ]:
#adata = scipy.io.mmread("/data/input_data/matrix.mtx")
#cells = pd.read_csv("/data/input_data/barcodes.tsv",sep="\t",header=None)
#genes = pd.read_csv("/data/input_data/features.tsv",sep="\t",header=None)

adata = sc.read_10x_mtx(
    '/data/INCF_10X_data/',  # the directory with the `.mtx` file
    var_names='gene_symbols',                # use gene symbols for the variable names (variables-axis index)
    cache=True)                              # write a cache file for faster subsequent reading

In [ ]:
adata.var_names_make_unique()

In [ ]:
# Show those genes that yield the highest fraction of counts in each single cell, across all cells.

sc.pl.highest_expr_genes(
    adata,
    n_top=20,
)

In [ ]:
# Basic Filtering

sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata.var["mt"] = adata.var_names.str.startswith(
    "MT-"
)  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(
    adata, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
)

In [ ]:
sc.pl.violin(
    adata,
    ["n_genes_by_counts", "total_counts", "pct_counts_mt"],
    jitter=0.4,
    multi_panel=True,
)

In [ ]:
# Remove cells that have too many mitochondrial genes expressed or too many total counts:

sc.pl.scatter(adata, x="total_counts", y="pct_counts_mt")
sc.pl.scatter(adata, x="total_counts", y="n_genes_by_counts")

In [ ]:
# Actually do the filtering

adata = adata[adata.obs.n_genes_by_counts < 2500, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

In [ ]:
# Total-count normalize (library-size correct) the data matrix $\mathbf{X}$ to 10,000 reads per cell, so that counts become comparable among cells.

sc.pp.normalize_total(adata, target_sum=1e4)

In [ ]:
# Logarithmize the data:

sc.pp.log1p(adata)

In [ ]:
# Identify highly-variable genes.

sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)

Set the `.raw` attribute of the AnnData object to the normalized and logarithmized raw gene expression for later use in differential testing and visualizations of gene expression. This simply freezes the state of the AnnData object.

In [ ]:
adata.raw = adata

In [ ]:
# Actually do the filtering

adata = adata[:, adata.var.highly_variable]

In [ ]:
# Regress out effects of total counts per cell and the percentage of mitochondrial genes expressed. Scale the data to unit variance.

sc.pp.regress_out(adata, ["total_counts", "pct_counts_mt"])

In [ ]:
# Scale each gene to unit variance. Clip values exceeding standard deviation 10.

sc.pp.scale(adata, max_value=10)

## Principal component analysis

In [ ]:
# Reduce the dimensionality of the data by running principal component analysis (PCA), which reveals the main axes of variation and denoises the data.

sc.tl.pca(adata, svd_solver="arpack")

In [ ]:
sc.pl.pca(adata, color='AURKAIP1')

Let us inspect the contribution of single PCs to the total variance in the data. This gives us information about how many PCs we should consider in order to compute the neighborhood relations of cells, e.g. used in the clustering function  `sc.tl.louvain()` or tSNE `sc.tl.tsne()`. In our experience, often a rough estimate of the number of PCs does fine.

In [ ]:
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
adata.write(results_file)

In [ ]:
adata

## Computing the neighborhood graph

In [ ]:
# Let us compute the neighborhood graph of cells using the PCA representation of the data matrix. 
# You might simply use default values here. For the sake of reproducing Seurat's results, let's take the following values.

sc.pp.neighbors(adata, n_neighbors=10, n_pcs=40)

## Embedding the neighborhood graph

We suggest embedding the graph in two dimensions using UMAP ([McInnes et al., 2018](https://arxiv.org/abs/1802.03426)), see below. It is potentially more faithful to the global connectivity of the manifold than tSNE, i.e., it better preserves trajectories. In some ocassions, you might still observe disconnected clusters and similar connectivity violations. They can usually be remedied by running:

```Python
sc.tl.paga(adata)
sc.pl.paga(adata, plot=False)  # remove `plot=False` if you want to see the coarse-grained graph
sc.tl.umap(adata, init_pos="paga")
```

In [ ]:
sc.tl.umap(adata)

In [ ]:
# adata.var["gene_ids"][10:25]


In [ ]:
sc.pl.umap(adata, color=["C1QL3", "FOXP2", "RORA"])

In [ ]:
sc.pl.umap(adata, color=["C1QL3", "FOXP2", "RORA"], use_raw=False)

## Clustering the neighborhood graph

In [ ]:
sc.tl.leiden(adata)

In [ ]:
sc.pl.umap(adata, color=["leiden", "C1QL3", "FOXP2", "RORA"])

In [ ]:
# Save the result

adata.write(results_file)

#### Let us compute a ranking for the highly differential genes in each cluster. For this, by default, the `.raw` attribute of AnnData is used in case it has been initialized before. The simplest and fastest method to do so is the t-test.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='t-test')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

The result of a [Wilcoxon rank-sum (Mann-Whitney-U)](https://en.wikipedia.org/wiki/Mann%E2%80%93Whitney_U_test) test is very similar. We recommend using the latter in publications, see e.g., [Sonison & Robinson (2018)](https://doi.org/10.1038/nmeth.4612). You might also consider much more powerful differential testing packages like MAST, limma, DESeq2 and, for python, the recent diffxpy.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
adata.write(results_file)

As an alternative, let us rank genes using logistic regression. For instance, this has been suggested by [Natranos et al. (2018)](https://doi.org/10.1101/258566). The essential difference is that here, we use a multi-variate appraoch whereas conventional differential tests are uni-variate. [Clark et al. (2014)](https://doi.org/10.1186/1471-2105-15-79) has more details.

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='logreg')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
# Reload the object that has been save with the Wilcoxon Rank-Sum test result.

adata = sc.read(results_file)

In [ ]:
# Show the 15 top ranked genes per cluster 0, 1, ..., 14 in a dataframe.

pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(15)


In [ ]:
# Get a table with the scores and groups.

result = adata.uns["rank_genes_groups"]
groups = result["names"].dtype.names
pd.DataFrame(
    {
        group + "_" + key[:1]: result[key][group]
        for group in groups
        for key in ["names", "pvals"]
    }
).head(15)

In [ ]:
# Compare to a single cluster:

sc.tl.rank_genes_groups(adata, "leiden", groups=["0"], reference="1", method="wilcoxon")
sc.pl.rank_genes_groups(adata, groups=["0"], n_genes=20)

If we want a more detailed view for a certain group, use `sc.pl.rank_genes_groups_violin`.

In [ ]:
sc.pl.rank_genes_groups_violin(adata, groups="0", n_genes=8)

In [ ]:
# Reload the object with the computed differential expression (i.e. DE via a comparison with the rest of the groups):

adata = sc.read(results_file)

In [ ]:
sc.pl.rank_genes_groups_violin(adata, groups="0", n_genes=8)

In [ ]:
# If you want to compare a certain gene across groups, use the following.

sc.pl.violin(adata, ["C1QL3", "FOXP2", "RORA"], groupby="leiden")

In [ ]:
adata

In [ ]:
adata.write(
    results_file, compression="gzip"
)  # `compression="gzip"` saves disk space, but slows down writing and subsequent reading

If you want to share this file with people who merely want to use it for visualization, a simple way to reduce the file size is by removing the dense scaled and corrected data matrix. The file still contains the raw data used in the visualizations in `adata.raw`.

In [ ]:
adata.raw.to_adata().write(f"{results_dir}/write/INCF_withoutX.h5ad")